## Introduction au NLP


Dans ce notebook, nous allons faire un projet en utilisant le Natural Language Processing, un domaine du machine learning qui consiste traiter ,analyser le langage naturel. Ce projet est inspiré d'un projet disponible dans le [site](https://www.actuia.com/contribution/victorbigand/tutoriel-tal-pour-les-debutants-classification-de-texte/).

### bibliothèques

In [ ]:
# verifier si un gpu est disponible
import tensorflow as tf
tf.test.gpu_device_name()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
import pandas as pd
import string
import nltk
from sklearn.model_selection import train_test_split

# for machine learning
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# for NLP
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

`TfidfVectorizer` est un outil utilisé pour transformer du texte brut en une matrice de caractéristiques (ou matrice de termes), où chaque ligne représente un document et chaque colonne représente un mot unique dans tout l'ensemble de documents. Elle utilise la technique TF-IDF (Term Frequency-Inverse Document Frequency).

**TF-IDF (Term Frequency - Inverse Document Frequency)** :
- Term Frequency (TF) : Mesure la fréquence d'un terme dans un document.
- Inverse Document Frequency (IDF) : Mesure l'importance d'un terme dans l'ensemble du corpus (documents). Plus un terme apparaît dans peu de documents, plus il est considéré comme important.

### Chargement de données:

In [ ]:
df1 = pd.read_csv('Youtube01-Psy.csv')
df2 = pd.read_csv('Youtube02-KatyPerry.csv')
df3 = pd.read_csv('Youtube03-LMFAO.csv')
df4 = pd.read_csv('Youtube04-Eminem.csv')
df5 = pd.read_csv('Youtube05-Shakira.csv')

df = pd.concat([df1,df2,df3,df4,df5])
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Youtube01-Psy.csv'

In [ ]:
df = df[['CONTENT','CLASS']].copy()
df['CLASS'].value_counts()

Quelques commentaires pris au hasard

In [ ]:
df.sample(5)

Les étapes courantes de nettoyage des données textuelles sont :

- Normalisation : Conversion en minuscules, suppression des accents et des espaces inutiles.
- Tokenisation : Division du texte en tokens (mots ou sous-mots).
- Suppression des stopwords : Élimination des mots peu informatifs (ex. : prépositions, articles).
- Lemmatisation / Stemming : Réduction des mots à leur forme de base ou à leur racine.
- N-grams : Création de séquences de n mots consécutifs pour capturer les relations contextuelles.

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
lemma = WordNetLemmatizer()
def lemmatize(sent):
    tokens = word_tokenize(sent.lower()) # Normalisation
    tokens = [lemma.lemmatize(lemma.lemmatize(lemma.lemmatize(w, 'v'), 'n'), 'a') for w in tokens]

    return ' '.join(tokens)

In [ ]:
df['CONTENT'] = df['CONTENT'].apply(lemmatize)


## Classification:

In [ ]:
vect = TfidfVectorizer(stop_words='english', analyzer='word', ngram_range=(1,2))

In [ ]:
tfidf_mat = vect.fit_transform(df['CONTENT'])
feature_names = vect.get_feature_names_out() # pour obtenir les mots des tokens
dense = tfidf_mat.todense() # Convertir en numpy array
denselist = dense.tolist() # convertir array en liste
df_tfidf = pd.DataFrame(denselist, columns=feature_names)
df2 = pd.DataFrame(denselist, columns=feature_names) # convertir en DataFrame
df2.head()

## Support vector machine:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df2, df['CLASS'], test_size=0.4, random_state=42)
clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(f1_score(y_test, y_pred, average=None))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.show()